# Showcasing IMAGE_META package
The GPS_WRITER_SHOWCASE notbook provides numerous manipulation features for manipulating jpg Image metadata leveraging the great [EXIF Tool](https://exiftool.org/) and using open street map geo API https://nominatim.org/release-docs/develop/api/Overview/ for getting geo meta data. 
The package contains the following modules:

* **geo.py** coordinate calculations, access to nominatim API for reverse geo encoding (coordinates to site plain text information), gpx file handling
* **persistence.py** reading + writing plain + json files
* **exif.py** exiftool interface + image metadata handling / transformation 
* **util** datetime calculations, binary search in list, ...

Caveat: Mind the usage terms from Nominatim https://operations.osmfoundation.org/policies/nominatim/ ! So reverse search is only accceptable for a small amount of requests!

## Exiftool Command Lines
You need to install exiftool and set path variables accordingly to be able to execute it in target directory. Find some examples her, for more info check out the following sources:

* **[EXIFTOOL FAQ](https://exiftool.org/faq.html 'EXIFTOOL FAQ')**
* **[EXIFTOOL EXAMPLES](https://exiftool.org/examples.html 'EXIFTOOL EXAMPLES')**
* **[EXIFTOOL DOCUMENTATION](https://exiftool.org/exiftool_pod.html 'EXIFTOOL DOCUMENTATION')**
* **[EXIFTOOL GEOTAGGING](https://exiftool.org/geotag.html 'EXIFTOOL GEOTAGGING')**

In [1]:
# here's import of all packages required to execute below examples
import os
from importlib import reload
from datetime import datetime
import pytz

import image_meta
import image_meta.persistence
import image_meta.util
import image_meta.geo
reload(image_meta)
reload(image_meta.persistence)
reload(image_meta.util)
reload(image_meta.geo)

# Import classes
from image_meta.persistence import Persistence as P
from image_meta.util import Util as U
from image_meta.geo import Geo as G
from image_meta.exif import ExifTool as E

# Sample Data
coords = {"Stuttgart":{"lat":48.7835,"lon":9.1850},
          "Tübingen":{"lat":48.52027,"lon":19.05361}}
lat,lon = list(coords["Tübingen"].values())
# OSM Link can be constructed like
print(f"Tübingen OSM Link -> https://www.openstreetmap.org/#map=15/{lat}/{lon}")
print("Reverse Search link:")
# Reverse Search url for this link is (click to see the data)
print(f"""https://nominatim.openstreetmap.org/reverse?format=jsonv2&lat={lat}\
&lon={lon}&addressdetails=16&namedetails=1&extratags=1""")
# timezone
tz_local = pytz.timezone("Europe/Berlin")
tz_utc = pytz.timezone("UTC")

Tübingen OSM Link -> https://www.openstreetmap.org/#map=15/48.52027/19.05361
Reverse Search link:
https://nominatim.openstreetmap.org/reverse?format=jsonv2&lat=48.52027&lon=19.05361&addressdetails=16&namedetails=1&extratags=1


In [2]:
## Exiftool Command Line Examples

In [3]:
import os

curr_path = os.path.abspath(os.getcwd())
#get test.jpg samples in samples subdirectory
sample_file = os.path.join(curr_path, "samples","test.jpg")
sample_json = os.path.join(curr_path, "samples","test.json")
if os.path.isfile(sample_file):
    # exiftool needs to be installed and available at command line in work dir
    print("--- Output of all EXIF subsegment metadata Containing date information ---")
    !exiftool -G -s -exif:*date* {sample_file}
    print("--- Same Data as json ---") 
    !exiftool -G -s -j -exif:*date* {sample_file}
    print("--- Output into json file (in samples folder ) ---")
    !exiftool -G -s -j -exif:*date* {sample_file} > {sample_json}
    #now we can read the json into dict
    print("reading from json into dict:")
    metadata_dict = P.read_json(sample_json)
    print(metadata_dict)

--- Output of all EXIF subsegment metadata Containing date information ---
[EXIF]          ModifyDate                      : 2016:05:25 12:47:47
[EXIF]          DateTimeOriginal                : 2016:05:19 14:18:15
[EXIF]          CreateDate                      : 2016:05:19 14:18:15
--- Same Data as json ---
[{
  "SourceFile": "C:/30_Entwicklung/WORK_JUPYTER/2020_EXIF_GPS_WRITER/samples/test.jpg",
  "EXIF:ModifyDate": "2016:05:25 12:47:47",
  "EXIF:DateTimeOriginal": "2016:05:19 14:18:15",
  "EXIF:CreateDate": "2016:05:19 14:18:15"
}]
--- Output into json file (in samples folder ) ---
reading from json into dict:
[{'SourceFile': 'C:/30_Entwicklung/WORK_JUPYTER/2020_EXIF_GPS_WRITER/samples/test.jpg', 'EXIF:ModifyDate': '2016:05:25 12:47:47', 'EXIF:DateTimeOriginal': '2016:05:19 14:18:15', 'EXIF:CreateDate': '2016:05:19 14:18:15'}]


# Geo Module

In [4]:
# convert lat lon into cartesian (X,Y,Z) coordinates
c1 = list(coords["Tübingen"].values())
G.latlon2cartesian(c1)

(1377.5860144550647, 3988.6725545919794, 4773.09009186109)

In [5]:
# calculate the distance of two coordinates (to initialize data, run the first cell above)
c1 = list(coords["Tübingen"].values())
c2 = list(coords["Stuttgart"].values())
G.get_distance(c1,c2,debug=True)

Arc Distance: 725.0202527688491 Distance: 724.6290934699693  Difference: 0.39115929887975653
Delta Coordinates (X,Y,Z): [707.5071314006651, -155.39330286324957, -19.336585674878734] 
 Distance: 725.0202527688491


725.0202527688491

# Utils Module

### Datetime Conversion

In [6]:
# get UTC Timestamp from Date String conforming to format ####-##-##T##:##:##Z / (+/-)##:##  
now = datetime.now()
now_s = now.strftime("%Y-%m-%dT%H:%M:%SZ")
now_ts = U.get_timestamp(now_s)
print(f"Now DateTime {now} -> Now String: {now_s} -> UTC Timestamp {now_ts}")
#convert back from timestamp
utc_dt = tz_utc.localize(datetime.utcfromtimestamp(now_ts))
cet_dt = utc_dt.astimezone(tz_local)
print("Timestamp -> Datetime UTC",utc_dt," -> Datetime Local",cet_dt)
print("UTC Offset",cet_dt.utcoffset()," Timezone",cet_dt.tzinfo)

Now DateTime 2020-06-17 23:27:42.130186 -> Now String: 2020-06-17T23:27:42Z -> UTC Timestamp 1592436462
Timestamp -> Datetime UTC 2020-06-17 23:27:42+00:00  -> Datetime Local 2020-06-18 01:27:42+02:00
UTC Offset 2:00:00  Timezone Europe/Berlin


# Exif Module

### Metadata Hierarchy
In photo management programs you often can maintain tags as hierarchies and export them as text file. In this file, a hierarchy level is represented as tab character. From this, you can construct hierarchical meta tags (stored as XMP:HierarchicalSubject in image metadata). The following method will read a hierarchy metadata file and put them into a dict with the "leaf" tag as dict key. This way, you can maintain a hierarchy and automaticall get the hierachical meta tag by just maintaining the hierarchy in a text file.   

In [7]:
# read the meta file (needs to be UTF8)
curr_path = os.path.abspath(os.getcwd())
#get test hierarchy samples in samples subdirectory
sample_hier = os.path.join(curr_path, "samples","test_hier.txt")
if not os.path.isfile(sample_hier):
    print("{sample_hier} NOT FOUND")
    exit
    
lines = P.read_file(sample_hier)
print("-----------HIERARCHY-------------")
for line in lines:
    print(line.strip('\n'))
print("-----------OUTPUT-------------")
h_tag_dict = E.create_meta_hierarchy_tags(lines,debug=False)
print(h_tag_dict)
tag = "Abend"
print("-----------Example-------------")
print(f"Tag <{tag}> has hierarchical attribute <{h_tag_dict[tag]}>")

-----------HIERARCHY-------------
Architektur
	Anlagen
		Baustelle
		Brücke
	Gebäude
		Bücherei
		Ruine
			Römer
	Gebäudedetail
		Denkmal
		Grabstein
	Innenraum
		Zimmer
Mood
	Blurry
	Cold
	Colorful
	Tranquil
Zeiten
	Tageszeiten
		Abend
		Mittag
		Morgen
		Nacht
-----------OUTPUT-------------
{'Architektur': 'Architektur', 'Anlagen': 'Architektur|Anlagen', 'Baustelle': 'Architektur|Anlagen|Baustelle', 'Brücke': 'Architektur|Anlagen|Brücke', 'Gebäude': 'Architektur|Gebäude', 'Bücherei': 'Architektur|Gebäude|Bücherei', 'Ruine': 'Architektur|Gebäude|Ruine', 'Römer': 'Architektur|Gebäude|Ruine|Römer', 'Gebäudedetail': 'Architektur|Gebäudedetail', 'Denkmal': 'Architektur|Gebäudedetail|Denkmal', 'Grabstein': 'Architektur|Gebäudedetail|Grabstein', 'Innenraum': 'Architektur|Innenraum', 'Zimmer': 'Architektur|Innenraum|Zimmer', 'Mood': 'Mood', 'Blurry': 'Mood|Blurry', 'Cold': 'Mood|Cold', 'Colorful': 'Mood|Colorful', 'Tranquil': 'Mood|Tranquil', 'Zeiten': 'Zeiten', 'Tageszeiten': 'Zeiten|Tagesz

### Process Images with Exiftool
In Class `ExifTool` executable will be triggered by `execute` method receiving control parameters and file list. In the constructor the image folder and the path to the Exiftool executable needs to be supplied. 
Convenience wrapper methods for handling metadata are supplied and described here. 

In [ ]:
curr_path = os.path.abspath(os.getcwd())
sample_jpg = os.path.join(curr_path, "samples","test.jpg")
exif_tool_path = r"C:\<pathto>\EXIFTOOL" #file path for exiftool eg r"c:\Tools\Exif\exiftool.exe
exif_tool_loc = os.path.join(exif_tool_path,"exiftool.exe")
print("Exiftool: ",exif_tool)

if not os.path.isfile(exif_tool_loc):
    print("EXIFTOOL NOT FOUND")
    exit

if not os.path.isfile(sample_jpg):
    print(f"file {sample_jpg} NOT FOUND")
    exit

# important: needs to be handled with with    
with E(exif_tool_loc) as exiftool:
    #collects data of several files in one dictionary
    meta_dict = exiftool.get_metadata(sample_jpg)

file_list = meta_dict.keys()

for jpg_file in file_list:
    print(f"--- File {jpg_file} ---")    
    meta_list = meta_dict[jpg_file]
    for meta in meta_list:
        print(f"[{meta}] ->  {meta_list[meta]}")            